In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urljoin

In [2]:
def scrape_character_images(url):
    # Lấy tên nhân vật từ URL
    character_name = url.split('/')[-2]
    
    # Tạo thư mục Ảnh và thư mục con cho nhân vật
    save_dir = os.path.join("Ảnh", character_name)
    os.makedirs(save_dir, exist_ok=True)
    
    # Gửi yêu cầu HTTP và lấy nội dung trang
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Lỗi: Không thể truy cập URL. Mã lỗi: {response.status_code}")
        return
    
    # Phân tích HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm phần Card Images
    card_images_section = None
    for heading in soup.find_all(['h2', 'h3']):
        if heading.text.strip() == "Card Images[]" or "Card Images" in heading.text:
            card_images_section = heading
            break
    
    if not card_images_section:
        print("Không tìm thấy phần Card Images trên trang")
        return
    
    # Tìm container chứa ảnh sau tiêu đề Card Images
    image_container = card_images_section.find_next('div', class_='wikia-gallery')
    
    if not image_container:
        # Thử tìm tất cả các ảnh sau tiêu đề này
        next_section = card_images_section.find_next(['h2', 'h3'])
        images = []
        current = card_images_section.next_sibling
        
        while current and (next_section is None or current != next_section):
            if current.name == 'figure' or (hasattr(current, 'find_all') and current.find_all('img')):
                images.extend(current.find_all('img'))
            current = current.next_sibling
            
        if not images:
            print("Không tìm thấy ảnh nào trong phần Card Images")
            return
    else:
        images = image_container.find_all('img')
    
    # Tải và lưu các ảnh
    count = 0
    for img in images:
        # Lấy URL của ảnh gốc (có độ phân giải cao)
        img_url = img.get('src') or img.get('data-src')
        if img_url:
            # Chuyển sang URL có độ phân giải cao nếu có thể
            img_url = re.sub(r'/scale-to-width-down/\d+', '', img_url)
            img_url = urljoin(url, img_url)
            
            # Lấy tên file
            filename = os.path.join(save_dir, f"{character_name}_card_{count}.jpg")
            
            try:
                # Tải ảnh
                img_data = requests.get(img_url).content
                
                # Lưu ảnh
                with open(filename, 'wb') as file:
                    file.write(img_data)
                
                print(f"Đã tải: {filename}")
                count += 1
            except Exception as e:
                print(f"Lỗi khi tải ảnh {img_url}: {str(e)}")
    
    print(f"Đã tải tổng cộng {count} ảnh cho nhân vật {character_name}")



In [3]:
# Ví dụ sử dụng
url = "https://projectsekai.fandom.com/wiki/Hatsune_Miku/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Hatsune_Miku\Hatsune_Miku_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hatsune_Miku\Hatsune_Miku_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hatsune_Miku\Hatsune_Miku_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hatsune_Miku\Hatsune_Miku_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No conn

In [4]:
url = "https://projectsekai.fandom.com/wiki/Kagamine_Rin/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Kagamine_Rin\Kagamine_Rin_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kagamine_Rin\Kagamine_Rin_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kagamine_Rin\Kagamine_Rin_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kagamine_Rin\Kagamine_Rin_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No conn

In [5]:
url = "https://projectsekai.fandom.com/wiki/Kagamine_Len/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Kagamine_Len\Kagamine_Len_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kagamine_Len\Kagamine_Len_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kagamine_Len\Kagamine_Len_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kagamine_Len\Kagamine_Len_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No conn

In [6]:
url = "https://projectsekai.fandom.com/wiki/Megurine_Luka/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Megurine_Luka\Megurine_Luka_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Megurine_Luka\Megurine_Luka_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Megurine_Luka\Megurine_Luka_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Megurine_Luka\Megurine_Luka_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D:

In [7]:
url = "https://projectsekai.fandom.com/wiki/MEIKO/Cards"
scrape_character_images(url)

Đã tải: Ảnh\MEIKO\MEIKO_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\MEIKO\MEIKO_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\MEIKO\MEIKO_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\MEIKO\MEIKO_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0

In [8]:
url = "https://projectsekai.fandom.com/wiki/KAITO/Cards"
scrape_character_images(url)

Đã tải: Ảnh\KAITO\KAITO_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\KAITO\KAITO_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\KAITO\KAITO_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\KAITO\KAITO_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0

In [9]:
url = "https://projectsekai.fandom.com/wiki/Hoshino_Ichika/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Hoshino_Ichika\Hoshino_Ichika_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hoshino_Ichika\Hoshino_Ichika_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hoshino_Ichika\Hoshino_Ichika_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hoshino_Ichika\Hoshino_Ichika_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAO

In [10]:
url = "https://projectsekai.fandom.com/wiki/Tenma_Saki/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Tenma_Saki\Tenma_Saki_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Tenma_Saki\Tenma_Saki_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Tenma_Saki\Tenma_Saki_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Tenma_Saki\Tenma_Saki_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters 

In [11]:
url = "https://projectsekai.fandom.com/wiki/Mochizuki_Honami/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Mochizuki_Honami\Mochizuki_Honami_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Mochizuki_Honami\Mochizuki_Honami_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Mochizuki_Honami\Mochizuki_Honami_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Mochizuki_Honami\Mochizuki_Honami_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAA

In [12]:
url = "https://projectsekai.fandom.com/wiki/Hinomori_Shiho/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Hinomori_Shiho\Hinomori_Shiho_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hinomori_Shiho\Hinomori_Shiho_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hinomori_Shiho\Hinomori_Shiho_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hinomori_Shiho\Hinomori_Shiho_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAO

In [13]:
url = "https://projectsekai.fandom.com/wiki/Hanasato_Minori/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Hanasato_Minori\Hanasato_Minori_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hanasato_Minori\Hanasato_Minori_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hanasato_Minori\Hanasato_Minori_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hanasato_Minori\Hanasato_Minori_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQ

In [3]:
url = "https://projectsekai.fandom.com/wiki/Kiritani_Haruka/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Kiritani_Haruka\Kiritani_Haruka_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kiritani_Haruka\Kiritani_Haruka_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kiritani_Haruka\Kiritani_Haruka_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kiritani_Haruka\Kiritani_Haruka_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQ

In [16]:

url = "https://projectsekai.fandom.com/wiki/Hinomori_Shizuku/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Hinomori_Shizuku\Hinomori_Shizuku_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hinomori_Shizuku\Hinomori_Shizuku_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hinomori_Shizuku\Hinomori_Shizuku_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Hinomori_Shizuku\Hinomori_Shizuku_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAA

In [17]:

url = "https://projectsekai.fandom.com/wiki/Momoi_Airi/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Momoi_Airi\Momoi_Airi_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Momoi_Airi\Momoi_Airi_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Momoi_Airi\Momoi_Airi_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Momoi_Airi\Momoi_Airi_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters 

In [18]:

url = "https://projectsekai.fandom.com/wiki/Azusawa_Kohane/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Azusawa_Kohane\Azusawa_Kohane_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Azusawa_Kohane\Azusawa_Kohane_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Azusawa_Kohane\Azusawa_Kohane_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Azusawa_Kohane\Azusawa_Kohane_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAO

In [19]:
url = "https://projectsekai.fandom.com/wiki/Shiraishi_An/Cards"
scrape_character_images(url)


Đã tải: Ảnh\Shiraishi_An\Shiraishi_An_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shiraishi_An\Shiraishi_An_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shiraishi_An\Shiraishi_An_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shiraishi_An\Shiraishi_An_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No conn

In [20]:

url = "https://projectsekai.fandom.com/wiki/Shinonome_Akito/Cards"
scrape_character_images(url)


Đã tải: Ảnh\Shinonome_Akito\Shinonome_Akito_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shinonome_Akito\Shinonome_Akito_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shinonome_Akito\Shinonome_Akito_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shinonome_Akito\Shinonome_Akito_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQ

In [21]:

url = "https://projectsekai.fandom.com/wiki/Aoyagi_Toya/Cards"
scrape_character_images(url)


Đã tải: Ảnh\Aoyagi_Toya\Aoyagi_Toya_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Aoyagi_Toya\Aoyagi_Toya_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Aoyagi_Toya\Aoyagi_Toya_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Aoyagi_Toya\Aoyagi_Toya_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection a

In [22]:
url = "https://projectsekai.fandom.com/wiki/Tenma_Tsukasa/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Tenma_Tsukasa\Tenma_Tsukasa_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Tenma_Tsukasa\Tenma_Tsukasa_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Tenma_Tsukasa\Tenma_Tsukasa_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Tenma_Tsukasa\Tenma_Tsukasa_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D:

In [23]:
url = "https://projectsekai.fandom.com/wiki/Otori_Emu/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Otori_Emu\Otori_Emu_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Otori_Emu\Otori_Emu_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Otori_Emu\Otori_Emu_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Otori_Emu\Otori_Emu_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were fou

In [24]:
url = "https://projectsekai.fandom.com/wiki/Kusanagi_Nene/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Kusanagi_Nene\Kusanagi_Nene_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kusanagi_Nene\Kusanagi_Nene_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kusanagi_Nene\Kusanagi_Nene_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kusanagi_Nene\Kusanagi_Nene_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D:

In [25]:
url = "https://projectsekai.fandom.com/wiki/Kamishiro_Rui/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Kamishiro_Rui\Kamishiro_Rui_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kamishiro_Rui\Kamishiro_Rui_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kamishiro_Rui\Kamishiro_Rui_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Kamishiro_Rui\Kamishiro_Rui_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D:

In [26]:
url = "https://projectsekai.fandom.com/wiki/Yoisaki_Kanade/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Yoisaki_Kanade\Yoisaki_Kanade_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Yoisaki_Kanade\Yoisaki_Kanade_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Yoisaki_Kanade\Yoisaki_Kanade_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Yoisaki_Kanade\Yoisaki_Kanade_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAO

In [7]:
url = "https://projectsekai.fandom.com/wiki/Asahina_Mafuyu/Cards"
scrape_character_images(url)

Không tìm thấy phần Card Images trên trang


In [28]:
url = "https://projectsekai.fandom.com/wiki/Shinonome_Ena/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Shinonome_Ena\Shinonome_Ena_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shinonome_Ena\Shinonome_Ena_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shinonome_Ena\Shinonome_Ena_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Shinonome_Ena\Shinonome_Ena_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D:

In [29]:
url = "https://projectsekai.fandom.com/wiki/Akiyama_Mizuki/Cards"
scrape_character_images(url)

Đã tải: Ảnh\Akiyama_Mizuki\Akiyama_Mizuki_card_0.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Akiyama_Mizuki\Akiyama_Mizuki_card_1.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Akiyama_Mizuki\Akiyama_Mizuki_card_2.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D: No connection adapters were found for 'data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAOw%3D%3D'
Đã tải: Ảnh\Akiyama_Mizuki\Akiyama_Mizuki_card_3.jpg
Lỗi khi tải ảnh data:image/gif;base64,R0lGODlhAQABAIABAAAAAP///yH5BAEAAAEALAAAAAABAAEAQAICTAEAO

In [7]:
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urljoin
import time

In [8]:
def scrape_transparent_card_trims(url):
    # Lấy tên nhân vật từ URL
    character_name = url.split('/')[-2]
    
    # Tạo thư mục Ảnh và thư mục con cho nhân vật
    save_dir = os.path.join("Ảnh", character_name)
    os.makedirs(save_dir, exist_ok=True)
    
    # Gửi yêu cầu HTTP và lấy nội dung trang
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Lỗi: Không thể truy cập URL. Mã lỗi: {response.status_code}")
        return 0
    
    # Phân tích HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm phần Transparent Card Trims bằng cách sử dụng class và id chính xác
    trims_section = soup.find('span', class_='mw-headline', id='Transparent_Card_Trims')
    
    if not trims_section:
        print(f"Không tìm thấy phần Transparent Card Trims trên trang của {character_name}")
        return 0
    
    # Lấy thẻ h2 cha
    trims_heading = trims_section.parent
    
    # Tìm tất cả các thẻ figure sau heading đến heading tiếp theo
    images = []
    current = trims_heading.find_next_sibling()
    
    while current and not (current.name == 'h2' or current.name == 'h3'):
        if current.name == 'figure' or (hasattr(current, 'find_all') and current.find_all('figure')):
            if current.name == 'figure':
                images.append(current)
            else:
                figures = current.find_all('figure')
                for figure in figures:
                    images.append(figure)
        current = current.find_next_sibling()
    
    # Kiểm tra nếu chưa tìm thấy hình ảnh, thử các phương pháp khác
    if not images:
        # Tìm tất cả thẻ figure trong div.gallery sau heading
        gallery_div = None
        current = trims_heading.find_next_sibling()
        while current and not (current.name == 'h2' or current.name == 'h3'):
            if current.name == 'div' and ('gallery' in current.get('class', [])):
                gallery_div = current
                break
            current = current.find_next_sibling()
            
        if gallery_div:
            images = gallery_div.find_all('figure')
    
    # Tải và lưu các ảnh
    count = 0
    for figure in images:
        img = figure.find('img')
        caption_elem = figure.find('figcaption')
        
        if not img or not img.get('src'):
            continue
            
        # Lấy URL của ảnh gốc
        img_url = img.get('src') or img.get('data-src')
        
        # Trích xuất url ảnh gốc (full size)
        if '/scale-to-width-down/' in img_url:
            img_url = re.sub(r'/scale-to-width-down/\d+', '', img_url)
        if '/thumb/' in img_url:
            img_url = re.sub(r'/thumb(/.*)/[^/]+$', r'\1', img_url)
            
        img_url = urljoin(url, img_url)
        
        # Xử lý tên file
        caption = caption_elem.text.strip() if caption_elem else f"{character_name}_card_{count}"
        filename = re.sub(r'[\\/*?:"<>|]', '_', caption)  # Loại bỏ ký tự không hợp lệ
        filename = f"{filename}.png"
        save_path = os.path.join(save_dir, filename)
        
        try:
            # Tải ảnh
            img_response = requests.get(img_url, headers=headers)
            if img_response.status_code == 200:
                # Lưu ảnh
                with open(save_path, 'wb') as file:
                    file.write(img_response.content)
                
                print(f"Đã tải: {save_path}")
                count += 1
            else:
                print(f"Không thể tải ảnh từ URL: {img_url}")
        except Exception as e:
            print(f"Lỗi khi tải ảnh {img_url}: {str(e)}")
    
    print(f"Đã tải tổng cộng {count} ảnh Transparent Card Trims cho nhân vật {character_name}")
    return count

In [9]:
def scrape_specified_characters():
    characters_to_scrape = [
        "Hatsune_Miku",
        "Kagamine_Rin",
        "Kagamine_Len",
        "Megurine_Luka",
        "MEIKO",
        "KAITO",
        "Hoshino_Ichika",
        "Tenma_Saki",
        "Mochizuki_Honami",
        "Hinomori_Shiho",
        "Hanasato_Minori",
        "Kiritani_Haruka",
        "Momoi_Airi",
        "Hinomori_Shizuku",
        "Azusawa_Kohane",
        "Shiraishi_An",
        "Shinonome_Akito",
        "Aoyagi_Toya",
        "Tenma_Tsukasa",
        "Otori_Emu",
        "Kusanagi_Nene",
        "Kamishiro_Rui",
        "Yoisaki_Kanade",
        "Asahina_Mafuyu",
        "Shinonome_Ena",
        "Akiyama_Mizuki"
    ]

    for character in characters_to_scrape:
        gallery_url = f"https://projectsekai.fandom.com/wiki/{character}/Gallery"
        print(f"Đang xử lý nhân vật: {character}")
        try:
            count = scrape_transparent_card_trims(gallery_url)
            print(f"Đã tải tổng cộng {count} ảnh Transparent Card Trims cho nhân vật {character}")
        except Exception as e:
            print(f"Lỗi khi xử lý nhân vật {character}: {str(e)}")
        print("-" * 50)
        # Tạm dừng để tránh quá tải server
        time.sleep(1)

In [9]:
import os

def count_files_in_folders(root_dir):
    """
    Counts the number of files in each subfolder of the given root directory and calculates the total file count.

    Args:
        root_dir (str): Path to the root directory containing categorical folders.

    Returns:
        dict: A dictionary where keys are folder names and values are file counts.
        int: Total number of files across all folders.
    """
    folder_counts = {}
    total_file_count = 0  # Initialize total file count

    # Iterate through all items in the root directory
    for folder_name in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder_name)

        # Check if the item is a directory
        if os.path.isdir(folder_path):
            # Count the number of files in the folder
            file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
            folder_counts[folder_name] = file_count

            # Add to the total file count
            total_file_count += file_count

    return folder_counts, total_file_count

# Example usage
extracted_dataset_path = "Ảnh"
counts, total_count = count_files_in_folders(extracted_dataset_path)

# Print the results
print("File counts per category:")
for folder, count in counts.items():
    print(f"{folder}: {count} files")

print(f"\nTotal number of files: {total_count}")

File counts per category:
Akiyama_Mizuki: 69 files
Aoyagi_Toya: 73 files
Asahina_Mafuyu: 72 files
Azusawa_Kohane: 72 files
Hanasato_Minori: 69 files
Hatsune_Miku: 84 files
Hinomori_Shiho: 74 files
Hinomori_Shizuku: 70 files
Hoshino_Ichika: 71 files
Kagamine_Len: 74 files
Kagamine_Rin: 75 files
KAITO: 74 files
Kamishiro_Rui: 70 files
Kiritani_Haruka: 24 files
Kusanagi_Nene: 68 files
Megurine_Luka: 73 files
MEIKO: 72 files
Mochizuki_Honami: 72 files
Momoi_Airi: 70 files
Otori_Emu: 69 files
Shinonome_Akito: 75 files
Shinonome_Ena: 71 files
Shiraishi_An: 70 files
Tenma_Saki: 75 files
Tenma_Tsukasa: 71 files
Yoisaki_Kanade: 71 files

Total number of files: 1828
